In [1]:
# | default_exp tools.task_monitor

In [4]:
# | export
"""
Task Monitor Tool - Tracks task execution and validates success criteria
"""
from typing import Dict, Any, List
from agentic.tools.base import Tool

class TaskMonitorTool(Tool):
    def __init__(self):
        super().__init__()
        self.active_tasks = {}
        self.completed_tasks = {}
        
    def get_tool_schema(self) -> Dict[str, Any]:
        return {
            "type": "function",
            "function": {
                "name": "task_monitor",
                "description": "Monitor task execution progress and validate success criteria.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "action": {
                            "type": "string",
                            "enum": ["start", "update", "complete", "validate", "status"],
                            "description": "Action to perform"
                        },
                        "task_id": {
                            "type": "string",
                            "description": "Task identifier"
                        },
                        "task_name": {
                            "type": "string",
                            "description": "Task name"
                        },
                        "success_criteria": {
                            "type": "string",
                            "description": "Success criteria to validate against"
                        },
                        "current_status": {
                            "type": "string",
                            "description": "Current task status or output"
                        },
                        "evidence": {
                            "type": "array",
                            "items": {"type": "string"},
                            "description": "Evidence of task completion"
                        }
                    },
                    "required": ["action"]
                }
            }
        }
    
    def execute(self, action: str, task_id: str = None, task_name: str = None, 
                success_criteria: str = None, current_status: str = None, 
                evidence: List[str] = None) -> Dict[str, Any]:
        """Execute task monitoring action"""
        
        if action == "start":
            if not task_id or not task_name:
                return {"status": "error", "message": "task_id and task_name required for start"}
            
            self.active_tasks[task_id] = {
                "name": task_name,
                "success_criteria": success_criteria,
                "status": "in_progress",
                "evidence": []
            }
            return {
                "status": "success",
                "message": f"Started monitoring task {task_id}: {task_name}",
                "active_tasks": len(self.active_tasks)
            }
        
        elif action == "update":
            if task_id not in self.active_tasks:
                return {"status": "error", "message": f"Task {task_id} not found"}
            
            if current_status:
                self.active_tasks[task_id]["status"] = current_status
            if evidence:
                self.active_tasks[task_id]["evidence"].extend(evidence)
            
            return {
                "status": "success",
                "message": f"Updated task {task_id}",
                "task_status": self.active_tasks[task_id]["status"]
            }
        
        elif action == "validate":
            if task_id not in self.active_tasks:
                return {"status": "error", "message": f"Task {task_id} not found"}
            
            task = self.active_tasks[task_id]
            criteria = task.get("success_criteria", success_criteria)
            
            if not criteria:
                return {"status": "error", "message": "No success criteria defined"}
            
            # Simple validation based on evidence and status
            evidence_list = task.get("evidence", evidence or [])
            validation_score = len(evidence_list) * 0.3  # Basic scoring
            
            if task["status"] == "completed" and evidence_list:
                validation_score += 0.7
            
            is_valid = validation_score >= 0.8
            
            return {
                "status": "success",
                "task_id": task_id,
                "validation_result": "passed" if is_valid else "failed",
                "validation_score": validation_score,
                "success_criteria": criteria,
                "evidence_count": len(evidence_list),
                "message": f"Task {task_id} validation {'passed' if is_valid else 'failed'}"
            }
        
        elif action == "complete":
            if task_id not in self.active_tasks:
                return {"status": "error", "message": f"Task {task_id} not found"}
            
            task = self.active_tasks.pop(task_id)
            task["status"] = "completed"
            if evidence:
                task["evidence"].extend(evidence)
            
            self.completed_tasks[task_id] = task
            
            return {
                "status": "success",
                "message": f"Completed task {task_id}: {task['name']}",
                "completed_tasks": len(self.completed_tasks),
                "active_tasks": len(self.active_tasks)
            }
        
        elif action == "status":
            return {
                "status": "success",
                "active_tasks": {k: {"name": v["name"], "status": v["status"]} for k, v in self.active_tasks.items()},
                "completed_tasks": {k: {"name": v["name"], "evidence_count": len(v["evidence"])} for k, v in self.completed_tasks.items()},
                "summary": f"{len(self.active_tasks)} active, {len(self.completed_tasks)} completed"
            }
        
        else:
            return {"status": "error", "message": f"Unknown action: {action}"}
